# Strategy (strategia)

# Przykład: logging.Formater

In [1]:
import logging

logger = logging.getLogger('strategy_logger')
handler = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.handlers = []
logger.addHandler(handler)
logger.error('error message')

2017-11-08 11:21:18,877 - strategy_logger - ERROR - error message


In [2]:
another_formatter = logging.Formatter('%(message)s')
handler.setFormatter(another_formatter)
logger.error('error message')

error message


# Ćwiczenie: zniżki

In [ ]:
from collections import namedtuple


class Customer(object):
    def __init__(self, name, fidelity):
        self.name = name
        self.fidelity = fidelity


class LineItem(object):
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity

    
class Order(object):  # Context
    def __init__(self, customer, cart, promotion = None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        return sum(item.total() for item in self.cart)

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())


class Promotion(object):  # Strategy: an abstract base class
    def discount(self, order):
        """Return discount as a positive dollar amount
        
        Return 0 if the discount is not applicable."""
        raise NotImplementedError
        

### Kod początkowy

In [123]:
class FidelityPromo(Promotion):
    """5% discount for customers with 1000 or more fidelity points"""
    
class BulkItemPromo(Promotion):
    """10% discount for each LineItem with 20 or more units"""
    
class LargeOrderPromo(Promotion):
    """7% discount for orders with 10 or more distinct items"""

### Rozwiązanie

### Oczekiwane zachowanie

Fidelity points - when customer has more than 1000 fidelity points discount of 5% is applied

In [125]:
cart = [LineItem('banana', 4, .5), 
        LineItem('apple', 10, 1.5), 
        LineItem('watermellon', 5, 5.0)]
john = Customer('John Doe', 0)
Order(john, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [126]:
ann = Customer('Ann Smith', 1100)
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

Bulk purchase - when customer buys more than 20 units of one item 10% discount is applied

In [127]:
bulk_cart = [LineItem('banana', 30, .5), LineItem('apple', 10, 1.5)]

In [128]:
Order(john, bulk_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

Large order - when customer makes long order with more than 10 distinct items 7% discount is applied

In [129]:
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]

In [130]:
Order(john, long_order, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [131]:
Order(john, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>